In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q transformers==4.36.0 datasets==2.16.0 peft==0.7.0 accelerate==0.25.0 torch==2.1.0 sentencepiece==0.1.99

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from peft import get_peft_model, LoraConfig, TaskType

class Config:
    BASE_MODEL = "Salesforce/codet5p-220m"
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.1
    EPOCHS = 2
    BATCH_SIZE = 8
    LEARNING_RATE = 3e-4
    MAX_LENGTH = 512
    DATASET_NAME = "b-mc2/sql-create-context"
    TRAIN_SAMPLES = 10000
    OUTPUT_DIR = "/kaggle/working/fine_tuned_text2sql_codet5"
    GRADIENT_ACCUMULATION = 2

config = Config()


In [4]:
dataset = load_dataset(config.DATASET_NAME, split="train")
dataset = dataset.shuffle(seed=42).select(range(config.TRAIN_SAMPLES))
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset, eval_dataset = dataset["train"], dataset["test"]

tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(
    config.BASE_MODEL,
    torch_dtype=torch.float32,
    device_map="auto"
)

lora_config = LoraConfig(
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    target_modules=["SelfAttention.q", "SelfAttention.v"],
    lora_dropout=config.LORA_DROPOUT,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [6]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 589,824 || all params: 223,471,872 || trainable%: 0.2639365727423629


In [7]:
def preprocess_function(examples):
    inputs = [f"Schema: {ctx}\nQuestion: {q}\nSQL:" for ctx, q in zip(examples["context"], examples["question"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=config.MAX_LENGTH, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=config.MAX_LENGTH, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION,
    learning_rate=config.LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=f"{config.OUTPUT_DIR}/logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=False,
    bf16=False,
    predict_with_generate=True,
    generation_max_length=config.MAX_LENGTH,
    report_to="none"
)

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model(config.OUTPUT_DIR)
tokenizer.save_pretrained(config.OUTPUT_DIR)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,5.574900,5.102997
1,5.204700,5.098799


('/kaggle/working/fine_tuned_text2sql_codet5/tokenizer_config.json',
 '/kaggle/working/fine_tuned_text2sql_codet5/special_tokens_map.json',
 '/kaggle/working/fine_tuned_text2sql_codet5/vocab.json',
 '/kaggle/working/fine_tuned_text2sql_codet5/merges.txt',
 '/kaggle/working/fine_tuned_text2sql_codet5/added_tokens.json',
 '/kaggle/working/fine_tuned_text2sql_codet5/tokenizer.json')

In [12]:
test_cases = [
    {
        "schema": 'CREATE TABLE sales (region TEXT, revenue INTEGER, year INTEGER)',
        "question": "Show total revenue by region for 2024."
    },
    {
        "schema": 'CREATE TABLE customers (id INTEGER, name TEXT, city TEXT)',
        "question": "List all customers from Delhi."
    }
]

for test in test_cases:
    prompt = f"Schema: {test['schema']}\nQuestion: {test['question']}\nSQL:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Q: {test['question']}\nSQL: {sql_query}\n")

Q: Show total revenue by region for 2024.
SQL: SELECT COUNT(region) FROM sales WHERE revenue = "2024" AND year = 2024

Q: List all customers from Delhi.
SQL: SELECT id FROM customers WHERE name = "Delhi" AND city = "Delhi"



In [13]:
!cd /kaggle/working && zip -r fine_tuned_text2sql_codet5.zip fine_tuned_text2sql_codet5

  adding: fine_tuned_text2sql_codet5/ (stored 0%)
  adding: fine_tuned_text2sql_codet5/merges.txt (deflated 54%)
  adding: fine_tuned_text2sql_codet5/vocab.json (deflated 59%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/ (stored 0%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/merges.txt (deflated 54%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/vocab.json (deflated 59%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/special_tokens_map.json (deflated 97%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/tokenizer_config.json (deflated 94%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/training_args.bin (deflated 51%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/rng_state.pth (deflated 25%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/trainer_state.json (deflated 53%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/scheduler.pt (deflated 56%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/tokenizer.json (deflated 72%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/adapter_config.json (deflated 50%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/README.md (deflated 66%)
  adding: fine_tuned_text2sql_codet5/checkpoint-562/optimizer.pt (deflated 8%)
  adding: fine_tuned_text2sql_codet5/special_tokens_map.json (deflated 97%)
  adding: fine_tuned_text2sql_codet5/tokenizer_config.json (deflated 94%)
  adding: fine_tuned_text2sql_codet5/adapter_model.safetensors (deflated 7%)
  adding: fine_tuned_text2sql_codet5/training_args.bin (deflated 51%)
  adding: fine_tuned_text2sql_c